In [1]:
import pickle
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import time

In [2]:
def fix_size(element, x_axis_size):
    """
    Funkcja zwraca listę elementów o wymiarze x równym x_axis_size, a wymiarze y wziętym z argumentu element.
    Jeśli element jest zbyt krótki to wydłużamy,
    jeśli zbyt długi to kroimy na kawałki (być może ostatni będzie wydłużony),
    jeśli jest ok to zostawiamy.
    """
    if element.shape[0] < x_axis_size:
        padding_size = (x_axis_size - element.shape[0], element.shape[1])
        padding = np.zeros(padding_size)
        return [np.concatenate((padding, element))]
    
    elif element.shape[0] > x_axis_size:
        res = []
        i = 0
        while True:
            res += fix_size(element[x_axis_size*i:x_axis_size*(i+1)], x_axis_size)
            i += 1
            if x_axis_size*i >= element.shape[0]:
                break
        return res
    return [element]


def batch_generator(file_name, batch_size=64, x_axis_size=20, reshape_y=False):
    """
    Batche tworzone przez generator są wymiaru (batch_size, x_axis_size, długość wektora embeddingu).
    
    file_name: Ścieżka do pickle'a
    batch_size: liczba rekordów w batchu
    x_axis_size: wymiar "x" (otoczenia) każdego wystąpienia
    reshape_y: flaga czy zwracany y powinien być wymiaru (n,) (False) czy (n, 1) (True)
    """
    with open(file_name, 'rb') as f:
        pickle_data = pickle.load(f)
    
    data = []
    labels = []
    for label in pickle_data:
        for el in pickle_data[label]:
            curr_data = fix_size(el, x_axis_size)
            data += curr_data
            labels += [label] * len(curr_data)
            
    data = np.asarray(data)
    labels = np.asarray(labels)
    if reshape_y:
        labels = labels.reshape(-1, 1)
         
    i = 0
    while True:
        yield data[batch_size*i:batch_size*(i+1), :, :], labels[batch_size*i:batch_size*(i+1)]
        i += 1
        if batch_size*i >= data.shape[0]:
            break

In [3]:
class_mapping = {
    'politycy': 0,
    'dziennikarze': 1,
    'duchowni': 2,
    'aktorzy': 3,
    'lekarze': 4,
    'malarze': 5,
    'muzycy': 6,
    'poeci': 7,
    'prawnicy': 8,
    'sportowcy': 9
}


class EmbedDataset(Dataset):
    def __init__(self, file_name, mapping, x_axis_size=20, reshape_y=False):
        with open(file_name, 'rb') as f:
            pickle_data = pickle.load(f)
        data = []
        labels = []
        for label in pickle_data:
            for el in pickle_data[label]:
                curr_data = fix_size(el, x_axis_size)
                data += curr_data
                labels += [label] * len(curr_data)
        self.data = np.asarray(data)
        self.mapping = mapping
        self.inverse_mapping = {v: k for k,v in self.mapping.items()}
        self.labels = np.asarray([self.mapping[l] for l in labels])
#         if reshape_y:
#             self.labels = self.labels.reshape(-1, 1)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [4]:
unique_classes = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class LSTM(nn.Module):
    def __init__(self, hidden_dim, unique_classes, seq_len, embeding_dim):
        super().__init__()
        self.seq_len = seq_len
        self.embeding_dim = embeding_dim
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(self.embeding_dim, self.hidden_dim, batch_first = True)
        # self.linear = nn.Linear(hidden_dim * seq_len, unique_classes)
        self.linear = nn.Linear(hidden_dim, unique_classes)

    def forward(self, x):
        batch_size, _ , _ = x.shape
        # lstm_out, _ = self.lstm(x.view(self.seq_len, batch_size, -1))
        # outputs = self.linear(lstm_out.reshape(batch_size, -1))
        _, (final_hidden_state, _) = self.lstm(x) #, (h_0, c_0))
        outputs = self.linear(final_hidden_state.view(batch_size, -1))
        return outputs

In [6]:
def train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_validation, device):
    val_ba = []
    val_prec = []
    val_rec = []
    val_f1 = []
    val_matth = []

    for epoch in range(20):
        train_start_time = time.time()
        for i, (X_batch, y_batch) in enumerate(provider_train):
            X_batch = X_batch.long().to(device)
            y_batch = y_batch.long().to(device)

            optimizer.zero_grad()
            target_preds = model(X_batch)
            loss = torch.nn.functional.cross_entropy(target_preds, y_batch)

            loss.backward()
            optimizer.step()


        true_values = np.array([])
        pred_values = np.array([])
        for i, (X_batch, y_batch) in enumerate(provider_validation):
            X_batch = X_batch.long().to(device)
            y_batch = y_batch.long().to(device)

            pred_values = np.append(pred_values, model(X_batch).argmax(dim = 1).cpu().numpy())
            true_values = np.append(true_values, y_batch.cpu().numpy())
        val_ba.append(balanced_accuracy_score(true_values, pred_values, adjusted=False))
        val_prec.append(precision_score(true_values, pred_values, average = 'weighted'))
        val_rec.append(recall_score(true_values, pred_values, average = 'weighted'))
        val_f1.append(f1_score(true_values, pred_values, average = 'weighted'))
        val_matth.append(matthews_corrcoef(true_values, pred_values))
        print('| Epoch: {:3d} | Time: {:6.2f}s | Balanced Acc: {:5.2f} | Prec: {:5.2f} | Recall: {:5.2f} | F1: {:5.2f} | Matthews: {:5.2f} |'
                      .format(epoch+1, (time.time() - train_start_time), val_ba[epoch], val_prec[epoch], val_rec[epoch], val_f1[epoch], val_matth[epoch]))

    test_ba = []
    test_prec = []
    test_rec = []
    test_f1 = []
    test_matth = []
    true_values = np.array([])
    pred_values = np.array([])
    for i, (X_batch, y_batch) in enumerate(provider_test):
        X_batch = X_batch.long().to(device)
        y_batch = y_batch.long().to(device)

        pred_values = np.append(pred_values, model(X_batch).argmax(dim = 1).cpu().numpy())
        true_values = np.append(true_values, y_batch.cpu().numpy())

    test_ba = balanced_accuracy_score(true_values, pred_values, adjusted=False)
    test_prec = precision_score(true_values, pred_values, average = 'weighted')
    test_rec = recall_score(true_values, pred_values, average = 'weighted')
    test_f1 = f1_score(true_values, pred_values, average = 'weighted')
    test_matth = matthews_corrcoef(true_values, pred_values)
    print('| TEST SET | Balanced Acc: {:5.2f} | Prec: {:5.2f} | Recall: {:5.2f} | F1: {:5.2f} | Matthews: {:5.2f} |'
                      .format(test_ba, test_prec, test_rec, test_f1, test_matth))
    
    return test_ba, test_prec, test_rec, test_f1, test_matth

# BERT

In [57]:
embeding_dim = 768
seq_len = 32
hidden_dim = 64

In [58]:
device

'cuda'

In [59]:
model = LSTM(embeding_dim=embeding_dim, hidden_dim=hidden_dim, unique_classes=unique_classes, seq_len=seq_len).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-3)
loss = torch.nn.functional.binary_cross_entropy_with_logits


bert_gazeta_train_path = './embeddings/BERT/GAZETA_train.pickle'
bert_gazeta_test_path = './embeddings/BERT/GAZETA_test.pickle'
bert_gazeta_validation_path = './embeddings/BERT/GAZETA_validation.pickle'

data = EmbedDataset(bert_gazeta_train_path, class_mapping)
provider_train = DataLoader(data, batch_size=64, shuffle=True)

data = EmbedDataset(bert_gazeta_test_path, class_mapping)
provider_test = DataLoader(data, batch_size=64, shuffle=False)

data = EmbedDataset(bert_gazeta_validation_path, class_mapping)
provider_validation = DataLoader(data, batch_size=64, shuffle=False)

train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_validation, device)

| Epoch:   1 | Time:   1.47s | Balanced Acc:  0.10 | Prec:  0.32 | Recall:  0.57 | F1:  0.41 | Matthews:  0.00 |
| Epoch:   2 | Time:   1.27s | Balanced Acc:  0.10 | Prec:  0.32 | Recall:  0.57 | F1:  0.41 | Matthews:  0.00 |
| Epoch:   3 | Time:   0.94s | Balanced Acc:  0.10 | Prec:  0.38 | Recall:  0.57 | F1:  0.42 | Matthews:  0.03 |
| Epoch:   4 | Time:   1.19s | Balanced Acc:  0.16 | Prec:  0.50 | Recall:  0.61 | F1:  0.53 | Matthews:  0.27 |
| Epoch:   5 | Time:   1.34s | Balanced Acc:  0.16 | Prec:  0.49 | Recall:  0.61 | F1:  0.52 | Matthews:  0.26 |
| Epoch:   6 | Time:   1.19s | Balanced Acc:  0.16 | Prec:  0.51 | Recall:  0.61 | F1:  0.51 | Matthews:  0.27 |
| Epoch:   7 | Time:   1.17s | Balanced Acc:  0.17 | Prec:  0.49 | Recall:  0.60 | F1:  0.52 | Matthews:  0.25 |
| Epoch:   8 | Time:   1.17s | Balanced Acc:  0.21 | Prec:  0.49 | Recall:  0.60 | F1:  0.51 | Matthews:  0.24 |
| Epoch:   9 | Time:   1.21s | Balanced Acc:  0.22 | Prec:  0.52 | Recall:  0.62 | F1:  0.54 | M

(0.217920061560746,
 0.5578550636863685,
 0.6009389671361502,
 0.5743093388299376,
 0.27992581357414636)

In [61]:
model = LSTM(embeding_dim=embeding_dim, hidden_dim=hidden_dim, unique_classes=unique_classes, seq_len=seq_len).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-3)
loss = torch.nn.functional.binary_cross_entropy_with_logits

bert_gazeta_train_path = './embeddings/BERT/ONET_train.pickle'
bert_gazeta_test_path = './embeddings/BERT/ONET_test.pickle'
bert_gazeta_validation_path = './embeddings/BERT/ONET_validation.pickle'

data = EmbedDataset(bert_gazeta_train_path, class_mapping)
provider_train = DataLoader(data, batch_size=64, shuffle=True)

data = EmbedDataset(bert_gazeta_test_path, class_mapping)
provider_test = DataLoader(data, batch_size=64, shuffle=False)

data = EmbedDataset(bert_gazeta_validation_path, class_mapping)
provider_validation = DataLoader(data, batch_size=64, shuffle=False)

train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_validation, device)

| Epoch:   1 | Time:   2.89s | Balanced Acc:  0.10 | Prec:  0.53 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   2 | Time:   2.72s | Balanced Acc:  0.10 | Prec:  0.53 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   3 | Time:   3.49s | Balanced Acc:  0.10 | Prec:  0.53 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   4 | Time:   2.35s | Balanced Acc:  0.13 | Prec:  0.59 | Recall:  0.74 | F1:  0.64 | Matthews:  0.16 |
| Epoch:   5 | Time:   2.13s | Balanced Acc:  0.13 | Prec:  0.58 | Recall:  0.73 | F1:  0.64 | Matthews:  0.14 |
| Epoch:   6 | Time:   1.87s | Balanced Acc:  0.21 | Prec:  0.60 | Recall:  0.73 | F1:  0.65 | Matthews:  0.20 |
| Epoch:   7 | Time:   2.46s | Balanced Acc:  0.18 | Prec:  0.61 | Recall:  0.74 | F1:  0.66 | Matthews:  0.23 |
| Epoch:   8 | Time:   2.58s | Balanced Acc:  0.16 | Prec:  0.60 | Recall:  0.72 | F1:  0.65 | Matthews:  0.17 |
| Epoch:   9 | Time:   2.21s | Balanced Acc:  0.16 | Prec:  0.59 | Recall:  0.72 | F1:  0.64 | M

(0.2404132760833132,
 0.5925585658863395,
 0.6828793774319066,
 0.6108894707642167,
 0.1909797456107152)

# random

In [62]:
embeding_dim = 4
seq_len = 32
hidden_dim = 64

In [63]:
model = LSTM(embeding_dim=embeding_dim, hidden_dim=hidden_dim, unique_classes=unique_classes, seq_len=seq_len).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-3)
loss = torch.nn.functional.binary_cross_entropy_with_logits


bert_gazeta_train_path = './embeddings/random/GAZETA_train.pickle'
bert_gazeta_test_path = './embeddings/random/GAZETA_test.pickle'
bert_gazeta_validation_path = './embeddings/random/GAZETA_validation.pickle'

data = EmbedDataset(bert_gazeta_train_path, class_mapping)
provider_train = DataLoader(data, batch_size=64, shuffle=True)

data = EmbedDataset(bert_gazeta_test_path, class_mapping)
provider_test = DataLoader(data, batch_size=64, shuffle=False)

data = EmbedDataset(bert_gazeta_validation_path, class_mapping)
provider_validation = DataLoader(data, batch_size=64, shuffle=False)

x,y = train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_validation, device)

| Epoch:   1 | Time:   0.96s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   2 | Time:   1.12s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   3 | Time:   1.03s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   4 | Time:   0.98s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   5 | Time:   1.03s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   6 | Time:   1.09s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   7 | Time:   1.08s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   8 | Time:   1.39s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | Matthews:  0.00 |
| Epoch:   9 | Time:   1.39s | Balanced Acc:  0.10 | Prec:  0.35 | Recall:  0.59 | F1:  0.44 | M

ValueError: too many values to unpack (expected 2)

In [64]:
model = LSTM(embeding_dim=embeding_dim, hidden_dim=hidden_dim, unique_classes=unique_classes, seq_len=seq_len).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-3)
loss = torch.nn.functional.binary_cross_entropy_with_logits

bert_gazeta_train_path = './embeddings/random/ONET_train.pickle'
bert_gazeta_test_path = './embeddings/random/ONET_test.pickle'
bert_gazeta_validation_path = './embeddings/random/ONET_validation.pickle'

data = EmbedDataset(bert_gafzeta_train_path, class_mapping)
provider_train = DataLoader(data, batch_size=64, shuffle=True)

data = EmbedDataset(bert_gazeta_test_path, class_mapping)
provider_test = DataLoader(data, batch_size=64, shuffle=False)

data = EmbedDataset(bert_gazeta_validation_path, class_mapping)
provider_validation = DataLoader(data, batch_size=64, shuffle=False)

train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_validation, device)

| Epoch:   1 | Time:   1.76s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   2 | Time:   1.62s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   3 | Time:   1.50s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   4 | Time:   1.77s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   5 | Time:   1.58s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   6 | Time:   1.69s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   7 | Time:   1.65s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   8 | Time:   2.39s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | Matthews:  0.00 |
| Epoch:   9 | Time:   1.63s | Balanced Acc:  0.10 | Prec:  0.54 | Recall:  0.73 | F1:  0.62 | M

(0.1111111111111111,
 0.4496360752046644,
 0.670549084858569,
 0.5383093250956242,
 0.0)

# Words
## GAZETA

In [7]:
def open_pickle(path):
    with open(path, 'rb') as f:
        pickle_data = pickle.load(f)
    return pickle_data
train = list(open_pickle('./embeddings/words/GAZETA_train.pickle').values())
test = list(open_pickle('./embeddings/words/GAZETA_test.pickle').values())
validation = list(open_pickle('./embeddings/words/GAZETA_validation.pickle').values())

In [8]:
words = set()
for i in train:
    for j in i:
            words = words.union(j)
for i in test:
    for j in i:
            words = words.union(j)
for i in validation:
    for j in i:
            words = words.union(j)
len(words)

28564

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(words))
le.transform(list(words))

array([ 4834, 17029, 22047, ...,  9605,  7812, 10669], dtype=int64)

In [10]:
class EmbedDatasetWords(Dataset):
    def __init__(self, pickle_data, mapping, x_axis_size=20, reshape_y=False):
        data = []
        labels = []
        for label in pickle_data:
            for el in pickle_data[label]:
                curr_data = fix_size(el, x_axis_size)
                data += curr_data
                labels += [label] * len(curr_data)
        
        self.data = np.asarray(data).reshape(len(data), x_axis_size)
        self.mapping = mapping
        self.inverse_mapping = {v: k for k,v in self.mapping.items()}
        self.labels = np.asarray([self.mapping[l] for l in labels])
#         if reshape_y:
#             self.labels = self.labels.reshape(-1, 1)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [11]:
with open('./embeddings/words/GAZETA_train.pickle', 'rb') as f:
    pickle_data = pickle.load(f)

train_data = {k: list() for k in pickle_data}
for label in pickle_data:
    for el in pickle_data[label]:
        converted = []
        converted = le.transform(el)
        train_data[label].append(np.array(converted).reshape(len(converted), 1))
        
data = EmbedDatasetWords(train_data, class_mapping)
provider_train = DataLoader(data, batch_size=64, shuffle=True)

with open('./embeddings/words/GAZETA_test.pickle', 'rb') as f:
    pickle_data = pickle.load(f)
    
test_data = {k: list() for k in pickle_data}
for label in pickle_data:
    for el in pickle_data[label]:
        converted = []
        converted = le.transform(el)
        test_data[label].append(np.array(converted).reshape(len(converted), 1))
        
data = EmbedDatasetWords(test_data, class_mapping)
provider_test = DataLoader(data, batch_size=64, shuffle=True)

with open('./embeddings/words/GAZETA_validation.pickle', 'rb') as f:
    pickle_data = pickle.load(f)
    
val_data = {k: list() for k in pickle_data}
for label in pickle_data:
    for el in pickle_data[label]:
        converted = []
        converted = le.transform(el)
        val_data[label].append(np.array(converted).reshape(len(converted), 1))
        
data = EmbedDatasetWords(val_data, class_mapping)
provider_val = DataLoader(data, batch_size=64, shuffle=True)

In [21]:
embeding_dim = 700
seq_len = 32
hidden_dim = 64

class LSTM(nn.Module):
    def __init__(self, hidden_dim, unique_classes, seq_len, embeding_dim, words):
        super().__init__()
        self.seq_len = seq_len
        self.embeding_dim = embeding_dim
        self.hidden_dim = hidden_dim
        
        self.embe = nn.Embedding(len(words), self.embeding_dim)
        self.lstm1 = nn.LSTM(self.embeding_dim, self.hidden_dim, batch_first = True)
        self.lstm2 = nn.LSTM(self.hidden_dim, self.hidden_dim, batch_first = True)
        # self.linear = nn.Linear(hidden_dim * seq_len, unique_classes)
        self.linear = nn.Linear(hidden_dim, unique_classes)

    def forward(self, x):
        batch_size, _ = x.shape
        # lstm_out, _ = self.lstm(x.view(self.seq_len, batch_size, -1))
        # outputs = self.linear(lstm_out.reshape(batch_size, -1))
        x = self.embe(x)
        _, (x, _) = self.lstm1(x) #, (h_0, c_0))
        _, (final_hidden_state, _) = self.lstm2(x) #, (h_0, c_0))
        outputs = self.linear(final_hidden_state.view(batch_size, -1))
        return outputs

In [22]:
unique_classes = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LSTM(64, 10, 20, 700, words).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-2)
loss = torch.nn.functional.binary_cross_entropy_with_logits

train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_val, device)

RuntimeError: size mismatch, m1: [64 x 1], m2: [64 x 10] at C:/w/1/s/tmp_conda_3.7_183424/conda/conda-bld/pytorch_1570818936694/work/aten/src\THC/generic/THCTensorMathBlas.cu:290

## ONET

In [14]:
def open_pickle(path):
    with open(path, 'rb') as f:
        pickle_data = pickle.load(f)
    return pickle_data
train = list(open_pickle('./embeddings/words/ONET_train.pickle').values())
test = list(open_pickle('./embeddings/words/ONET_test.pickle').values())
validation = list(open_pickle('./embeddings/words/ONET_validation.pickle').values())

In [15]:
words = set()
for i in train:
    for j in i:
            words = words.union(j)
for i in test:
    for j in i:
            words = words.union(j)
for i in validation:
    for j in i:
            words = words.union(j)
len(words)

28504

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(words))
le.transform(list(words))

array([ 4681, 26669, 15812, ...,  7470,  2620, 10318], dtype=int64)

In [17]:
with open('./embeddings/words/ONET_train.pickle', 'rb') as f:
    pickle_data = pickle.load(f)

train_data = {k: list() for k in pickle_data}
for label in pickle_data:
    for el in pickle_data[label]:
        converted = []
        converted = le.transform(el)
        train_data[label].append(np.array(converted).reshape(len(converted), 1))
        
data = EmbedDatasetWords(train_data, class_mapping)
provider_train = DataLoader(data, batch_size=64, shuffle=True)

with open('./embeddings/words/ONET_test.pickle', 'rb') as f:
    pickle_data = pickle.load(f)
    
test_data = {k: list() for k in pickle_data}
for label in pickle_data:
    for el in pickle_data[label]:
        converted = []
        converted = le.transform(el)
        test_data[label].append(np.array(converted).reshape(len(converted), 1))
        
data = EmbedDatasetWords(test_data, class_mapping)
provider_test = DataLoader(data, batch_size=64, shuffle=True)

with open('./embeddings/words/ONET_validation.pickle', 'rb') as f:
    pickle_data = pickle.load(f)
    
val_data = {k: list() for k in pickle_data}
for label in pickle_data:
    for el in pickle_data[label]:
        converted = []
        converted = le.transform(el)
        val_data[label].append(np.array(converted).reshape(len(converted), 1))
        
data = EmbedDatasetWords(val_data, class_mapping)
provider_val = DataLoader(data, batch_size=64, shuffle=True)

In [18]:
unique_classes = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LSTM(64, 10, 20, 700, words).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-2)
loss = torch.nn.functional.binary_cross_entropy_with_logits

train_result_all_xd(model, optimizer, loss, provider_train, provider_test, provider_val, device)

| Epoch:   1 | Time:   4.68s | Balanced Acc:  0.10 | Prec:  0.56 | Recall:  0.73 | F1:  0.62 | Matthews:  0.07 |
| Epoch:   2 | Time:   4.66s | Balanced Acc:  0.13 | Prec:  0.60 | Recall:  0.71 | F1:  0.63 | Matthews:  0.08 |
| Epoch:   3 | Time:   4.71s | Balanced Acc:  0.14 | Prec:  0.59 | Recall:  0.68 | F1:  0.62 | Matthews:  0.08 |
| Epoch:   4 | Time:   4.74s | Balanced Acc:  0.15 | Prec:  0.59 | Recall:  0.66 | F1:  0.62 | Matthews:  0.07 |
| Epoch:   5 | Time:   4.65s | Balanced Acc:  0.15 | Prec:  0.59 | Recall:  0.67 | F1:  0.62 | Matthews:  0.08 |
| Epoch:   6 | Time:   4.69s | Balanced Acc:  0.16 | Prec:  0.59 | Recall:  0.66 | F1:  0.62 | Matthews:  0.08 |
| Epoch:   7 | Time:   4.80s | Balanced Acc:  0.15 | Prec:  0.59 | Recall:  0.66 | F1:  0.62 | Matthews:  0.07 |
| Epoch:   8 | Time:   5.31s | Balanced Acc:  0.15 | Prec:  0.59 | Recall:  0.67 | F1:  0.62 | Matthews:  0.08 |
| Epoch:   9 | Time:   9.46s | Balanced Acc:  0.15 | Prec:  0.60 | Recall:  0.68 | F1:  0.62 | M

C:\Users\tomas\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1855: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\Users\tomas\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.1634172634940347,
 0.54901270055837,
 0.6014975041597338,
 0.5648433538078744,
 0.13605801501937742)